# AgriChat (TinyLlama) — Fast Agriculture Assistant

**Optimized for free Colab GPUs (T4).** This notebook fine-tunes TinyLlama 1.1B with LoRA and launches a Gradio chat UI with **short, concise replies (1–2 sentences)**.

---

## 1) Install dependencies

In [ ]:
!pip install -q --upgrade pip
!pip install -q transformers accelerate datasets peft bitsandbytes safetensors sentencepiece gradio

## 2) Create training script, dataset and accelerate config

In [ ]:
%%writefile train_lora.py
# Training script for LoRA fine-tuning (TinyLlama 1.1B)
import argparse, os
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, default_data_collator
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training


def make_prompt(example):
    return example['prompt'] + '\n' + example['response']


def tokenize_fn(examples, tokenizer, max_seq_length):
    texts = [make_prompt(x) for x in examples]
    tokenized = tokenizer(texts, truncation=True, max_length=max_seq_length, padding='max_length')
    input_ids = tokenized['input_ids']
    attention_mask = tokenized['attention_mask']
    labels = [ids.copy() for ids in input_ids]
    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}


def main():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model_name_or_path', type=str, required=True)
    parser.add_argument('--dataset_path', type=str, required=True)
    parser.add_argument('--output_dir', type=str, required=True)
    parser.add_argument('--micro_batch_size', type=int, default=4)
    parser.add_argument('--num_train_epochs', type=int, default=1)
    parser.add_argument('--max_seq_length', type=int, default=256)
    args = parser.parse_args()

    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=True)
    if tokenizer.pad_token_id is None:
        tokenizer.add_special_tokens({'pad_token': '<|pad|>'})

    print('Loading model (4-bit if supported)...')
    try:
        model = AutoModelForCausalLM.from_pretrained(args.model_name_or_path, load_in_4bit=True, device_map='auto', trust_remote_code=True)
        model = prepare_model_for_kbit_training(model)
    except Exception as e:
        print('4-bit loading failed, loading in default precision. Error:', e)
        model = AutoModelForCausalLM.from_pretrained(args.model_name_or_path, device_map='auto', trust_remote_code=True)

    lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=['q_proj','v_proj'], lora_dropout=0.05, bias='none', task_type='CAUSAL_LM')
    model = get_peft_model(model, lora_config)
    model.resize_token_embeddings(len(tokenizer))

    data = load_dataset('json', data_files=args.dataset_path)
    ds = data['train'].train_test_split(test_size=0.1) if len(data['train'])>1 else {'train': data['train'], 'test': data['train']}
    train_ds, val_ds = ds['train'], ds['test']

    def map_fn(examples):
        return tokenize_fn(examples, tokenizer, args.max_seq_length)
    train_ds = train_ds.map(map_fn, batched=True, remove_columns=train_ds.column_names)
    val_ds = val_ds.map(map_fn, batched=True, remove_columns=val_ds.column_names)

    training_args = TrainingArguments(
        output_dir=args.output_dir,
        per_device_train_batch_size=args.micro_batch_size,
        per_device_eval_batch_size=args.micro_batch_size,
        num_train_epochs=args.num_train_epochs,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        evaluation_strategy='steps',
        eval_steps=50,
        save_steps=200,
        save_total_limit=2,
        remove_unused_columns=False,
        report_to='none',
    )

    trainer = Trainer(model=model, args=training_args, train_dataset=train_ds, eval_dataset=val_ds, data_collator=default_data_collator)
    trainer.train()

    model.save_pretrained(os.path.join(args.output_dir, 'lora_adapter'))
    tokenizer.save_pretrained(args.output_dir)
    print('Training complete. Adapter saved to', os.path.join(args.output_dir, 'lora_adapter'))

if __name__ == '__main__':
    main()


In [ ]:
import json
examples = [{"prompt": "What is crop rotation?", "response": "Crop rotation improves soil health and reduces pests."}, {"prompt": "Explain organic farming.", "response": "Organic farming avoids synthetic fertilizers and relies on compost and natural pest control."}, {"prompt": "What are the types of irrigation systems?", "response": "Surface, sprinkler, and drip irrigation are common types."}, {"prompt": "What is soil erosion and how can we prevent it?", "response": "Soil erosion is the loss of topsoil by wind or water; prevent it with terraces, cover crops, and contour plowing."}, {"prompt": "What are hybrid seeds?", "response": "Hybrid seeds are bred from two varieties to get better yield and resistance."}, {"prompt": "Explain the importance of fertilizers.", "response": "Fertilizers supply essential nutrients like nitrogen, phosphorus, and potassium for plant growth."}, {"prompt": "What is the function of photosynthesis?", "response": "Photosynthesis converts sunlight into energy and oxygen for plants."}, {"prompt": "What are the advantages of drip irrigation?", "response": "Drip irrigation saves water and delivers nutrients directly to roots."}, {"prompt": "What is vermicomposting?", "response": "Vermicomposting uses earthworms to turn organic waste into rich fertilizer."}, {"prompt": "Explain the greenhouse effect.", "response": "The greenhouse effect traps heat in the atmosphere and keeps Earth warm."}, {"prompt": "Customer: Hi, my internet is not working.\\nAgent:", "response": "Please restart your router; if it still fails, share your connection details."}, {"prompt": "Customer: I was charged twice for my order.\\nAgent:", "response": "Sorry for that; please provide your order ID and we will check for a refund."}, {"prompt": "Customer: How can I track my package?\\nAgent:", "response": "Use the tracking link sent to your email or enter your order number on the website."}, {"prompt": "Customer: The product I received is damaged.\\nAgent:", "response": "We apologize; please upload a photo and we will arrange a replacement."}]
with open('instructions.jsonl','w', encoding='utf-8') as f:
    for e in examples:
        f.write(json.dumps(e, ensure_ascii=False) + '\n')
print('Wrote instructions.jsonl with', len(examples), 'entries')

In [ ]:
with open('accelerate_config.yaml','w') as f:
    f.write('''compute_environment: LOCAL_MACHINE\ndistributed_type: NO\nmixed_precision: fp16\nnum_machines: 1\nnum_processes: 1\n''')
print('Wrote accelerate_config.yaml')

## 3) Run training
Run this command to start training your LoRA adapter. TinyLlama runs comfortably on free Colab GPUs.

In [ ]:
!accelerate launch train_lora.py \
  --model_name_or_path TinyLlama/TinyLlama-1.1B-Chat-v1.0 \
  --dataset_path ./instructions.jsonl \
  --output_dir ./outputs \
  --micro_batch_size 4 \
  --num_train_epochs 2 \
  --max_seq_length 256

## 4) Save outputs to Google Drive (optional but recommended)
Mount Drive and copy outputs so your adapter is safe after the Colab session ends.

In [ ]:
from google.colab import drive
import os
print('Mounting Google Drive...')
drive.mount('/content/drive')
# create destination folder
os.makedirs('/content/drive/MyDrive/AgriChat_Tiny_outputs', exist_ok=True)
print('Drive mounted. Outputs will be copied after training.')

## 5) Gradio chat UI — AgriChat (short replies)
After training completes, run the cell below to start the Gradio app. It loads the base model and the LoRA adapter, and generates short replies (1–2 sentences).

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch

base = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0'
adapter_path = './outputs/lora_adapter'

def load_model():
    print('Loading base model...')
    model = AutoModelForCausalLM.from_pretrained(base, device_map='auto', trust_remote_code=True)
    print('Loading LoRA adapter...')
    model = PeftModel.from_pretrained(model, adapter_path)
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained(base)
    return tokenizer, model

# generation function tuned for short replies

def generate_reply(prompt, tokenizer, model):
    inp = prompt
    inputs = tokenizer(inp, return_tensors='pt', truncation=True, max_length=128).to('cuda')
    with torch.no_grad():
        out = model.generate(**inputs, max_new_tokens=60, do_sample=False, temperature=0.2)
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    # post-process: keep first 2 sentences
    sentences = text.split('.')
    short = '.'.join([s.strip() for s in sentences[:2] if s.strip()])
    if short and not short.endswith('.'):
        short = short + '.'
    return short

# build Gradio interface

tokenizer, model = None, None

def respond(user_input):
    global tokenizer, model
    if tokenizer is None or model is None:
        tokenizer, model = load_model()
    return generate_reply(user_input, tokenizer, model)

iface = gr.Interface(fn=respond, inputs=gr.Textbox(lines=2, placeholder='Ask AgriChat something...'), outputs=gr.Textbox(label='AgriChat — Short reply'), title='AgriChat — Your Smart Agriculture Assistant (TinyLlama)', description='Short, clear answers (1–2 sentences).')
iface.launch(share=False, inbrowser=False)


## 6) Optional: Copy outputs to Drive
Run this cell after training to save adapter and tokenizer to your Drive.

In [ ]:
import shutil
src = './outputs'
dst = '/content/drive/MyDrive/AgriChat_Tiny_outputs/outputs'
if os.path.exists(src):
    shutil.copytree(src, dst, dirs_exist_ok=True)
    print('✅ Outputs copied to', dst)
else:
    print('No outputs folder found. Run training first.')

### Final notes
- TinyLlama 1.1B is chosen for compatibility with free Colab GPUs.
- If you want more detailed replies later, switch back to Mistral and ensure you have a large GPU runtime.
- Tell me when training finishes and I can help you upload the adapter to Hugging Face.